In [1]:
%env CUDA_VISIBLE_DEVICES=''


from PIL import Image
import numpy as np
import json
import time
import glob
import cv2
import os
import matplotlib.pyplot as plt
from pprint import pprint

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

env: CUDA_VISIBLE_DEVICES=''


In [2]:
from utils_refactored import FileManager, EllipseParser, BoundingBox, convert_grayscale_to_rgb

In [3]:
def process_paths(content_files: list, just_names_files: list) -> dict:
    content = []
    just_names = []
    for cnt, nms in zip(content_files, just_names_files):
        content_ = FileManager.read_txt(path=cnt)
        just_names_ = FileManager.read_txt(path=nms)
        content.extend(content_)
        just_names.extend(just_names_)
    return EllipseParser.parse_doc(content, just_names)

content_files = glob.glob("../01-data/03-fddb-data/FDDB-folds/FDDB-folds/*-ellipseList.txt")
just_names_files = glob.glob("../01-data/03-fddb-data/FDDB-folds/FDDB-folds/*.txt")
just_names_files = [item for item in just_names_files if item not in content_files]

ellipse_data = process_paths(content_files, just_names_files)

In [4]:
def process_images(img_paths: dict, debug=False, save_examples=False) -> dict:
    coco_json = {
        "images": [],
        "annotations": []
    }

    for i, (image_id, img_data) in enumerate(img_paths.items()):
        img_path, img_key = img_data
        cpp_detections_path = "./outputs/full_range/.._" + img_path.replace("/", "_") + ".txt"
        detections = []
        
        with open(cpp_detections_path, 'r') as file:
            for line_number, line in enumerate(file, start=1):
                try:
                    # Split the line by comma and convert each value to float
                    xmin, ymin, xmax, ymax, confidence, class_id = map(float, line.strip().split(',')[:-1])
                    
                    # Append the detection to the detections list
                    detections.append([xmin, ymin, xmax, ymax, confidence, class_id])
                except ValueError as e:
                    print(f"Error on line {line_number}: {line.strip()}")
                    print(f"Exception: {e}")

        #load img
        image = Image.open(img_path)
        image = convert_grayscale_to_rgb(image)
        image_width, image_height = image.size
        image_info = {
            "id": image_id,
            "file_name": img_key,
            "width": image_width,
            "height": image_height
        }
        coco_json["images"].append(image_info)

        try:
            for det in detections:
                x1, y1, x2, y2, confidence, class_id = det            
                x1 = int(x1/192.0*image_width)
                x2 = int(x2/192.0*image_width)
                y1 = int(y1/192.0*image_height)
                y2 = int(y2/192.0*image_height)
                w = x2-x1
                h = y2-y1
                #cv2.rectangle(img_arr, (x1, y1), (x1+w, y1+h), (0, 255, 0), 2)

                coco_bbox = [x1, y1, w, h]
                detection = {
                    "image_id": image_id,
                    "category_id": 1,
                    "bbox": coco_bbox,
                    "score": confidence,
                    "iscrowd": 0,
                    "area": w * h,
                    "segmentation": [],
                }
                coco_json["annotations"].append(detection)
            if save_examples and i <= 1:
                plt.imshow(img_arr)
                plt.show()
        except Exception as e:
            print(e)
            continue
    return coco_json




In [5]:
coco_gt = COCO('./data/coco_ground_truths.json')
filename_to_id = {img['file_name']: img['id'] for img in coco_gt.dataset['images']}
root = "../01-data/03-fddb-data/originalPics/"
ext = '.jpg'
img_paths = {value: [os.path.join(root,key + ext),key] for key,value in filename_to_id.items()}



loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [6]:
import glob

def count_jpg_files(root_folder):
    return len(glob.glob(f'{root_folder}/**/*.jpg', recursive=True))

jpg_count = count_jpg_files(root)
print(f'There are {jpg_count} .jpg files in {root} and its subfolders.')


There are 28204 .jpg files in ../01-data/03-fddb-data/originalPics/ and its subfolders.


In [ ]:
coco_json = process_images(img_paths)

In [19]:
coco_save_name = './data/coco_detections_cpp_inference_full_range.json'

with open(coco_save_name, 'w') as f:
    json.dump(coco_json['annotations'], f)

### Sanity checks

In [7]:
def load_img_from_img_id(coco_gt,id):
    data = coco_gt.imgs[id]
    root = "../01-data/03-fddb-data/originalPics/"
    ext = '.jpg'
    path = os.path.join(root,data['file_name'] + ext)
    img = cv2.imread(path)[...,::-1]
    return img

def draw_bounding_boxes(img, gt_anns, pred_anns,plot=True):
    img_c = img.copy()
    # Draw ground truth annotations
    for ann in gt_anns:
        bbox = ann['bbox']
        x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        cv2.rectangle(img_c, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Green color for GT

    # Draw prediction annotations
    for ann in pred_anns:
        bbox = ann['bbox']
        x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        cv2.rectangle(img_c, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Red color for predictions

    if plot:
        plt.imshow(img_c)
        plt.show()
    return img_c

In [8]:
coco_dt = coco_gt.loadRes('./data/coco_detections_cpp_inference_full_range.json')
img_ids = coco_dt.getImgIds()
debug = False
save_examples = True
plot = False

Loading and preparing results...
DONE (t=9.95s)
creating index...
index created!


In [9]:
for _id in img_ids:
    img = load_img_from_img_id(coco_gt,_id)
    #print(img)
    # pred ids
    ann_ids = coco_dt.getAnnIds(imgIds=_id)
    pr_anns = coco_dt.loadAnns(ann_ids)

    # gt ids
    ann_ids = coco_gt.getAnnIds(imgIds=_id)
    gt_anns = coco_gt.loadAnns(ann_ids)
    if debug:
        print(img.shape)
        pprint(gt_anns[0])
        print("="*80)
        pprint(pr_anns)
    img_c = draw_bounding_boxes(img,gt_anns,pr_anns,plot=plot)
    if save_examples:
        save_root = "./outputs/examples_of_predictions_and_gts/"
        save_name = str(_id) + ".jpg"
        save_name = os.path.join(save_root,save_name)
        cv2.imwrite(save_name,img_c[...,::-1])